In [ ]:
#THIS FUNCTION IS FOR RUNNING WITH SLURM JOB ARRAY
#(SPLITS UP JOB_ARRAY BELOW INTO EVEN MORE TASKS)
def StartSlurmJobArray(num_jobs,num_slurm_jobs, ISRUN):
    job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    if job_id==0: job_id=1
    if ISRUN==False:
        start_job=1;end_job=num_jobs+1
        return start_job,end_job
    total_elements=num_jobs #total num of variables

    job_range = total_elements // num_slurm_jobs  # Base size for each chunk
    remaining = total_elements % num_slurm_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_slurm_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_slurm_jobs - 1: 
            end_job = total_elements 
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_slurm_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()

    # if sbatch==True:
        
    start_job, end_job = get_job_range(job_id, num_slurm_jobs)
    index_adjust=start_job
    # print(f'start_job = {start_job}, end_job = {end_job}')
    if start_job==0: start_job=1
    if end_job==total_elements: end_job+=1
    return start_job,end_job

# job_id=1
# [start_slurm_job,end_slurm_job,slurm_index_adjust]=StartSlurmJobArray(num_jobs,num_slurm_jobs,ISRUN)
# parcel=parcel1.isel(xh=slice(start_job,end_job))

In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

# Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

# # dx = 1 km; Np = 1M; Nt = 5 min
# data1=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc', decode_timedelta=True) #***
# parcel1=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc', decode_timedelta=True) #***
# res='1km';t_res='5min'
# Np_str='1e6'

# # dx = 1km; Np = 50M
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_1km_1min.nc', decode_timedelta=True) #***
# parcel1=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc', decode_timedelta=True) #***
# res='1km'; t_res='1min'; Np_str='50e6'

# # dx = 1km; Np = 50M; Nz = 95
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_1km_1min_95nz.nc', decode_timedelta=True) #***
# parcel1=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_95nz.nc', decode_timedelta=True) #***
# res='1km'; t_res='1min_95nz'; Np_str='50e6'

# dx = 250m; Np = 50M
#Importing Model Data
dir2='/home/air673/koa_scratch/'
data1=xr.open_dataset(dir2+'cm1out_250m_1min_50M.nc', decode_timedelta=True) #***
parcel1=xr.open_dataset(dir2+'cm1out_pdata_250m_1min_50M.nc', decode_timedelta=True) #***
res='250m'; t_res='1min'; Np_str='50e6'

In [ ]:
times=data1['time'].values/(1e9 * 60); times=times.astype(float);
minutes=1/times[1] #1 / minutes per timestep = timesteps per minute
kms=np.argmax(data1['xh'].values-data1['xh'][0].values >= 1)

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(PlottingFunctions, inspect.isfunction)]
# functions

In [ ]:
#READING BACK IN
########################
def LoadFinalData(in_file):
    dict = {}
    with h5py.File(in_file, 'r') as f:
        for key in f.keys():
            dict[key] = f[key][:]
    return dict

def LoadAllCloudBase():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/'
    in_file = dir2 + f"all_cloudbase_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        all_cloudbase = pickle.load(f)
    return(all_cloudbase)
min_all_cloudbase=np.nanmin(LoadAllCloudBase())
print(f"Minimum Cloudbase is: {min_all_cloudbase*1000:.0f} m")

def LoadMeanLFC():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/'
    in_file = dir2 + f"MeanLFC_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        MeanLFC = pickle.load(f)
    return MeanLFC
MeanLFC=LoadMeanLFC()
print(f"Mean LFC is: {MeanLFC:.0f} m\n")

dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/'
in_file=dir2+f"parcel_tracking_SUBSET_{res}_{t_res}_{Np_str}.h5"
final_dict=LoadFinalData(in_file)


#DYNAMICALLY CREATING VARIABLES
for key, value in final_dict.items():
    globals()[key] = value

# #DYNAMICALLY PRINTING VARIABLE SIZES
# for key in final_dict:
#     print(f"{key} has {final_dict[key].shape[0]} parcels")

# PRINTING VARIABLE SIZES (ONE BY ONE)
print(f'ALL: {len(CL_ALL_out_arr)} CL parcels and {len(nonCL_ALL_out_arr)} nonCL parcels')
print(f'SHALLOW: {len(CL_SHALLOW_out_arr)} CL parcels and {len(nonCL_SHALLOW_out_arr)} nonCL parcels')
print(f'DEEP: {len(CL_DEEP_out_arr)} CL parcels and {len(nonCL_DEEP_out_arr)} nonCL parcels')
print('\n')
print(f'ALL: {len(SBZ_ALL_out_arr)} SBZ parcels and {len(nonSBZ_ALL_out_arr)} nonSBZ parcels')
print(f'SHALLOW: {len(SBZ_SHALLOW_out_arr)} SBZ parcels and {len(nonSBZ_SHALLOW_out_arr)} nonSBZ parcels')
print(f'DEEP: {len(SBZ_DEEP_out_arr)} SBZ parcels and {len(nonSBZ_DEEP_out_arr)} nonSBZ parcels')
print('\n')
print(f'ALL: {len(ColdPool_ALL_out_arr)} ColdPool parcels')
print(f'SHALLOW: {len(ColdPool_SHALLOW_out_arr)} ColdPool parcels')
print(f'DEEP: {len(ColdPool_DEEP_out_arr)} ColdPool parcels')


def apply_job_array_filter(start_job, end_job):
    # print("APPLYING JOB ARRAY")

    def job_filter(arr):
        return arr[(arr[:, 0] >= start_job) & (arr[:, 0] < end_job)]

    target_names = [
        'CL_ALL_out_arr', 'nonCL_ALL_out_arr',
        'CL_SHALLOW_out_arr', 'nonCL_SHALLOW_out_arr',
        'CL_DEEP_out_arr', 'nonCL_DEEP_out_arr',
        'SBZ_ALL_out_arr', 'nonSBZ_ALL_out_arr',
        'SBZ_SHALLOW_out_arr', 'nonSBZ_SHALLOW_out_arr',
        'SBZ_DEEP_out_arr', 'nonSBZ_DEEP_out_arr',
        'ColdPool_ALL_out_arr', 'ColdPool_SHALLOW_out_arr', 'ColdPool_DEEP_out_arr'
    ]

    for name in target_names:
        globals()[name+'_2'] = job_filter(globals()[name])

In [ ]:
##############################################
#SETUP 

In [ ]:
# def MakeDictionary(**vars):
#     return vars
    
# def GetData(start_job,end_job):
#     dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
#     in_file=dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5'
    
#     var_names = ['W', 'QCQI']
#     data_dict = make_data_dict(in_file,var_names,read_type,start_job,end_job)
#     W, QCQI = (data_dict[k] for k in var_names)
    
#     # #Making Time Matrix
#     # rows, cols = A.shape[0], A.shape[1]
#     # T = np.arange(rows).reshape(-1, 1) * np.ones((1, cols), dtype=int)

#     return VARs

In [ ]:
################################
#DATA SETUP
################################
data_type="Tracked_Area" 

In [ ]:
################################
#JOB ARRAY SETUP
################################
#*#*
# how many total jobs are being run? i.e. array=1-100 ==> num_jobs=100
if Np_str=='1e6':
    num_jobs=180 #1M parcels
    num_slurm_jobs=60
if Np_str=='50e6':
    num_jobs=2100 #50M parcels
    num_slurm_jobs=300
##############################

In [ ]:
##############################################
#DATA LOADING FUNCTIONS

In [ ]:
#JOB ARRAY SETUP
def StartJobArray(job_id,num_jobs):
    total_elements=len(parcel1['xh']) #total num of variables

    if num_jobs >= total_elements:
        raise ValueError("Number of jobs cannot be greater than or equal to total elements.")
    
    job_range = total_elements // num_jobs  # Base size for each chunk
    remaining = total_elements % num_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_jobs - 1: 
            end_job = total_elements #- 1
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()

    # if sbatch==True:
    #     job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    #     if job_id==0: job_id=1
        
    start_job, end_job = get_job_range(job_id, num_jobs)
    index_adjust=start_job
    # print(f'start_job = {start_job}, end_job = {end_job}')
    return start_job,end_job,index_adjust

# job_id=1
# [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)
# parcel=parcel1.isel(xh=slice(start_job,end_job))

In [ ]:
# Reading Back Data Later
##############
def make_data_dict(in_file,var_names,read_type,start_job,end_job):
    if read_type=='h5py':
        with h5py.File(in_file, 'r') as f:
           data_dict = {var_name: f[var_name][:,start_job:end_job] for var_name in var_names}
            
    elif read_type=='xarray':
        in_data = xr.open_dataset(
            in_file,
            engine='h5netcdf',
            phony_dims='sort',
            chunks={'phony_dim_0': 100, 'phony_dim_1': 1_000_000} 
        )
        data_dict = {k: in_data[k][:,start_job:end_job].compute().data for k in var_names}
    return data_dict

# read_type='xarray'
read_type='h5py'

In [ ]:
def GetSpatialData(start_job,end_job):
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
    in_file=dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5'
    
    var_names = ['Z', 'Y', 'X', 'z']
    data_dict = make_data_dict(in_file,var_names,read_type,start_job,end_job)
    Z, Y, X, parcel_z = (data_dict[k] for k in var_names)
    
    # #Making Time Matrix
    # rows, cols = A.shape[0], A.shape[1]
    # T = np.arange(rows).reshape(-1, 1) * np.ones((1, cols), dtype=int)
    
    return Z,Y,X,parcel_z

In [ ]:
################################################################################

In [ ]:
##############################################
#MAKE PROFILES FUNCTIONS

In [ ]:
# #CALL_VARIABLES FUNCTION
def call_variables(t,z,y,x):
    t = t if t is not None else slice(None)
    z = z if z is not None else slice(None)
    y = y if y is not None else slice(None)
    x = x if x is not None else slice(None)
    
    variable='winterp'; w_data=data1[variable].isel(time=t,zh=z,yh=y,xh=x).data
    variable='qc'; qc_data=data1[variable].isel(time=t,zh=z,yh=y,xh=x).data
    variable='qi'; qi_data=data1[variable].isel(time=t,zh=z,yh=y,xh=x).data
    qc_plus_qi=qc_data+qi_data
    return w_data,qc_plus_qi

# def call_variables1(data1,t): #TESTING METHOD 2 (SLOWER)
#     variable='winterp'; w_data_t=data1[variable].isel(time=t).data
#     variable='qc'; qc_data_t=data1[variable].isel(time=t).data
#     variable='qi'; qi_data_t=data1[variable].isel(time=t).data
#     return w_data_t,qc_data_t,qi_data_t

# def call_variables2(w_data_t,qc_data_t,qi_data_t, z,y,x): #TESTING METHOD 2 (SLOWER)
#     z = z if z is not None else slice(None)
#     y = y if y is not None else slice(None)
#     x = x if x is not None else slice(None)
    
#     variable='winterp'; w_data=w_data_t[z,y,x]
#     variable='qc'; qc_data=qc_data_t[z,y,x]
#     variable='qi'; qi_data=qi_data_t[z,y,x]
#     qc_plus_qi=qc_data+qi_data
#     return w_data,qc_plus_qi

In [ ]:
def GetIndices(out_arr, Z, Y, X, index_adjust):
    all_ts = []; all_zs = []; all_ys = []; all_xs = []

    for row in range(out_arr.shape[0]):
        after=out_arr[row,3]
        if np.mod(row,50)==0: print(f'{row}/{out_arr.shape[0]}')
        p=out_arr[row,0]
        
        # ts=np.arange(out_arr[row,4],out_arr[row,5]+1 + after)
        ts_end = min(out_arr[row, 2] + 1 + after, len(data1['time'])) #this takes care of exceeding buffers
        ts = np.arange(out_arr[row, 1], ts_end)
        
        zs=Z[ts,p-index_adjust] #JOBARRAY INDEXADJUST
        ys=Y[ts,p-index_adjust] #JOBARRAY INDEXADJUST
        xs=X[ts,p-index_adjust] #JOBARRAY INDEXADJUST
        all_ts.extend(ts); all_zs.extend(zs); 
        all_ys.extend(ys);all_xs.extend(xs)

    ################################################################################
    # Convert to numpy arrays
    all_ts = np.array(all_ts, dtype=int); all_zs = np.array(all_zs, dtype=int)
    all_ys = np.array(all_ys, dtype=int); all_xs = np.array(all_xs, dtype=int)

    # Sort by time
    sort_order = np.argsort(all_ts)
    ts_sorted = all_ts[sort_order]; zs_sorted = all_zs[sort_order]
    ys_sorted = all_ys[sort_order]; xs_sorted = all_xs[sort_order]
    return ts_sorted, zs_sorted, ys_sorted, xs_sorted

In [ ]:
# #MAKING PROFILE ARRAY #METHOD 1 (DOUBLE FOR LOOP in p and (t,z,y,x)) (FASTEST CURRENTLY)

# #thresholds
# w_thresh1=0.1
# w_thresh2=0.5
# qcqi_thresh=1e-6

# dx=(data1['xf'][1].item()-data1['xf'][0].item())# km *1e3 #meters
# dy=(data1['yf'][1].item()-data1['yf'][0].item())# km *1e3 #meters

# def TrackedProfile_AREA_VERSION1(Z,Y,X,type,type2,updraft_type):
#     global index_adjust
#     out_arr = globals()[f"{type2}_{type.upper()}_out_arr"+"_2"].copy()
   
#     zhs=data1['zh'].values
#     profile_array =np.zeros((len(zhs), 3)) #column 1: var, column 2: counter, column 3: list of zhs
#     profile_array[:,2]=zhs;

#     #####
#     zhs=data1['zh'].values
#     profile_array_squares =np.zeros((len(zhs), 3)) #column 1: var, column 2: counter, column 3: list of zhs
#     profile_array_squares[:,2]=zhs;
#     #####
    
#     for row in range(out_arr.shape[0]):
#         after=out_arr[row,3]
#         if np.mod(row,50)==0: print(f'{row}/{out_arr.shape[0]}')
#         p=out_arr[row,0]
        
#         # ts=np.arange(out_arr[row,4],out_arr[row,5]+1 + after)
#         ts_end = min(out_arr[row, 2] + 1 + after, len(data1['time'])) #this takes care of exceeding buffers
#         ts = np.arange(out_arr[row, 1], ts_end)
        
#         zs=Z[ts,p-index_adjust] #JOBARRAY INDEXADJUST
#         ys=Y[ts,p-index_adjust] #JOBARRAY INDEXADJUST
#         xs=X[ts,p-index_adjust] #JOBARRAY INDEXADJUST

#         for t, z, y, x in zip(ts, zs, ys, xs):
#             #CALCULATING AREA (CURRENTLY IGNORES CASES WHERE UPDRAFT CROSSES THE BOUNDARY)
#             ########################################################
            
#             #FINDING XLENGTH
#             ########################################################
#             [x_slice,x_slice2]=call_variables(t,z,y,None)
            
#             if updraft_type=='general':
#                 x_slice = np.where((x_slice >= w_thresh1) & (x_slice2 < qcqi_thresh), 1, 0)
#             elif updraft_type=='cloudy':
#                 x_slice = np.where((x_slice >= w_thresh2) & (x_slice2 >= qcqi_thresh), 1, 0)
    
#             if np.all(x_slice[x+1:]==1) or np.all(x_slice[:(x-1)+1]==1): #*AVOID BOUNDARY CASES*
#                 continue
#             x_left=np.where(x_slice[:(x)+1]==0)[0][-1]
#             x_right=np.where(x_slice[x:]==0)[0][0]+(x)
#             x_length=(x_right-x_left)*dx
            
#             #FINDING YLENGTH
#             ########################################################
#             [y_slice,y_slice2]=call_variables(t,z,None,x)
            
#             if updraft_type=='general':
#                 y_slice = np.where((y_slice >= w_thresh1) & (y_slice2 < qcqi_thresh), 1, 0)
#             elif updraft_type=='cloudy':
#                 y_slice = np.where((y_slice >= w_thresh2) & (y_slice2 >= qcqi_thresh), 1, 0)
    
#             if np.all(y_slice[y+1:]==1) or np.all(y_slice[:(y-1)+1]==1): #*TO AVOID BOUNDARY CASES*
#                 continue
#             y_left=np.where(y_slice[:(y)+1]==0)[0][-1]
#             y_right=np.where(y_slice[y:]==0)[0][0]+(y)
#             y_length=(y_right-y_left)*dy

#             #CALCULATING AREA
#             ########################################################
#             AREA=x_length*y_length #square area approximation 
#             AREA*=np.pi/4 #include for oval area approximation

#             #ADDING TO PROFILE
#             ########################################################
#             var=AREA
#             profile_array[z,0]+=var;profile_array[z,1]+=1
#             profile_array_squares[z,0]+=var**2;profile_array_squares[z,1]+=1
#     return profile_array,profile_array_squares

In [ ]:
#MAKING PROFILE ARRAY #METHOD 2 (SINGLE FOR LOOP in (t,z,y,x)) (ONLY SLIGHLY SLOWER, BUT MAY BE FASTER IN THE LONG RUN)
#(SLIGHTLY DIFFERENT RESULTS FOR SOME Z LEVELS (count is the same), ORDER OF 1e-14, CAUSED BY SOME SUPER SLIGHT NUMERICAL DIFFERENCE FROM COMBINING FOR LOOP()

#thresholds
w_thresh1=0.1
w_thresh2=0.5
qcqi_thresh=1e-6

dx=(data1['xf'][1].item()-data1['xf'][0].item())# km *1e3 #meters
dy=(data1['yf'][1].item()-data1['yf'][0].item())# km *1e3 #meters

def TrackedProfile_AREA_VERSION2(Z,Y,X,type,type2,updraft_type):
    global index_adjust
    out_arr = globals()[f"{type2}_{type.upper()}_out_arr"+"_2"].copy()
   
    zhs=data1['zh'].values
    profile_array =np.zeros((len(zhs), 3)) #column 1: var, column 2: counter, column 3: list of zhs
    profile_array[:,2]=zhs;

    #####
    zhs=data1['zh'].values
    profile_array_squares =np.zeros((len(zhs), 3)) #column 1: var, column 2: counter, column 3: list of zhs
    profile_array_squares[:,2]=zhs;
    #####

    ts,zs,ys,xs=GetIndices(out_arr,Z,Y,X,index_adjust)
    for t, z, y, x in zip(ts, zs, ys, xs):
        if t % 5 == 25: print(f"{t}/{len(ts)}")
        #CALCULATING AREA (CURRENTLY IGNORES CASES WHERE UPDRAFT CROSSES THE BOUNDARY)
        ########################################################
        
        #FINDING XLENGTH
        ########################################################
        [x_slice,x_slice2]=call_variables(t,z,y,None)
        
        if updraft_type=='general':
            x_slice = np.where((x_slice >= w_thresh1) & (x_slice2 < qcqi_thresh), 1, 0)
        elif updraft_type=='cloudy':
            x_slice = np.where((x_slice >= w_thresh2) & (x_slice2 >= qcqi_thresh), 1, 0)

        if np.all(x_slice[x+1:]==1) or np.all(x_slice[:(x-1)+1]==1): #*AVOID BOUNDARY CASES*
            continue
        x_left=np.where(x_slice[:(x)+1]==0)[0][-1]
        x_right=np.where(x_slice[x:]==0)[0][0]+(x)
        x_length=(x_right-x_left)*dx
        
        #FINDING YLENGTH
        ########################################################
        [y_slice,y_slice2]=call_variables(t,z,None,x)
        
        if updraft_type=='general':
            y_slice = np.where((y_slice >= w_thresh1) & (y_slice2 < qcqi_thresh), 1, 0)
        elif updraft_type=='cloudy':
            y_slice = np.where((y_slice >= w_thresh2) & (y_slice2 >= qcqi_thresh), 1, 0)

        if np.all(y_slice[y+1:]==1) or np.all(y_slice[:(y-1)+1]==1): #*TO AVOID BOUNDARY CASES*
            continue
        y_left=np.where(y_slice[:(y)+1]==0)[0][-1]
        y_right=np.where(y_slice[y:]==0)[0][0]+(y)
        y_length=(y_right-y_left)*dy

        #CALCULATING AREA
        ########################################################
        AREA=x_length*y_length #square area approximation 
        AREA*=np.pi/4 #include for oval area approximation

        #ADDING TO PROFILE
        ########################################################
        var=AREA
        profile_array[z,0]+=var;profile_array[z,1]+=1
        profile_array_squares[z,0]+=var**2;profile_array_squares[z,1]+=1
    return profile_array,profile_array_squares

In [ ]:
# #MAKING PROFILE ARRAY #TESTING METHOD 3 (DOUBLE FOR LOOP in t_unique and (z,y,x)) (SLOWEST, LIKELY NOT IN THE LONG RUN EITHER)

# #thresholds
# w_thresh1=0.1
# w_thresh2=0.5
# qcqi_thresh=1e-6

# dx=(data1['xf'][1].item()-data1['xf'][0].item())# km *1e3 #meters
# dy=(data1['yf'][1].item()-data1['yf'][0].item())# km *1e3 #meters

# def TrackedProfile_AREA(Z,Y,X,type,type2,updraft_type):
#     global index_adjust
#     out_arr = globals()[f"{type2}_{type.upper()}_out_arr"+"_2"].copy()
   
#     zhs=data1['zh'].values
#     profile_array =np.zeros((len(zhs), 3)) #column 1: var, column 2: counter, column 3: list of zhs
#     profile_array[:,2]=zhs;
#     #####
#     zhs=data1['zh'].values
#     profile_array_squares =np.zeros((len(zhs), 3)) #column 1: var, column 2: counter, column 3: list of zhs
#     profile_array_squares[:,2]=zhs;
#     #####

#     ts,zs,ys,xs=GetIndices(out_arr,Z,Y,X,index_adjust) #GET ALL INDICES
#     for count,t in enumerate(np.unique(ts)): #LOOP THROUGH T OF UNIQUE T INDICES
#         if count % 30 == 0: print(f"{count}/{len(np.unique(ts))}")
#         w_data_t,qc_data_t,qi_data_t=call_variables1(data1,t) #CALL IN DATA AT EACH TIME
#         t_indices=np.where(ts==t); z_indices=zs[t_indices];y_indices=ys[t_indices];x_indices=xs[t_indices] #GET Z,Y,X, INDICIES FOR SPECIFIC TIME
#         for count, (z, y, x) in enumerate(zip(z_indices, y_indices, x_indices)):
#             #CALCULATING AREA (CURRENTLY IGNORES CASES WHERE UPDRAFT CROSSES THE BOUNDARY)
#             ########################################################
            
#             #FINDING XLENGTH
#             ########################################################
#             [x_slice,x_slice2]=call_variables2(w_data_t,qc_data_t,qi_data_t, z,y,None) #*#*#*#
            
#             if updraft_type=='general':
#                 x_slice = np.where((x_slice >= w_thresh1) & (x_slice2 < qcqi_thresh), 1, 0)
#             elif updraft_type=='cloudy':
#                 x_slice = np.where((x_slice >= w_thresh2) & (x_slice2 >= qcqi_thresh), 1, 0)
    
#             if np.all(x_slice[x+1:]==1) or np.all(x_slice[:(x-1)+1]==1): #*AVOID BOUNDARY CASES*
#                 continue
#             x_left=np.where(x_slice[:(x)+1]==0)[0][-1]
#             x_right=np.where(x_slice[x:]==0)[0][0]+(x)
#             x_length=(x_right-x_left)*dx
            
#             #FINDING YLENGTH
#             ########################################################
#             [y_slice,y_slice2]=call_variables2(w_data_t,qc_data_t,qi_data_t, z,None,x)
            
#             if updraft_type=='general':
#                 y_slice = np.where((y_slice >= w_thresh1) & (y_slice2 < qcqi_thresh), 1, 0)
#             elif updraft_type=='cloudy':
#                 y_slice = np.where((y_slice >= w_thresh2) & (y_slice2 >= qcqi_thresh), 1, 0)
    
#             if np.all(y_slice[y+1:]==1) or np.all(y_slice[:(y-1)+1]==1): #*TO AVOID BOUNDARY CASES*
#                 continue
#             y_left=np.where(y_slice[:(y)+1]==0)[0][-1]
#             y_right=np.where(y_slice[y:]==0)[0][0]+(y)
#             y_length=(y_right-y_left)*dy
    
#             #CALCULATING AREA
#             ########################################################
#             AREA=x_length*y_length #square area approximation 
#             AREA*=np.pi/4 #include for oval area approximation
    
#             #ADDING TO PROFILE
#             ########################################################
#             var=AREA
#             profile_array[z,0]+=var;profile_array[z,1]+=1
#             profile_array_squares[z,0]+=var**2;profile_array_squares[z,1]+=1
#     return profile_array,profile_array_squares

In [ ]:
def RunCalculation(Z,Y,X):
    type1_options = ['all', 'shallow', 'deep']
    type2_options = ['CL', 'nonCL', 'SBZ', 'nonSBZ', 'ColdPool']
    updraft_types = ['general', 'cloudy']

    Dictionary = {}

    for t1 in type1_options:
        print(f'{t1.upper()}')
        for t2 in type2_options:
            for utype in updraft_types:
                key = f"{t2}_{t1.upper()}_profile_array_{utype}"
                key_squares = f"{t2}_{t1.upper()}_profile_array_{utype}_squares"
                # [Dictionary[key],Dictionary[key_squares]] = TrackedProfile_AREA_VERSION1(Z, Y, X, type=t1, type2=t2, updraft_type=utype) #VERSION 1
                [Dictionary[key],Dictionary[key_squares]] = TrackedProfile_AREA_VERSION2(Z, Y, X, type=t1, type2=t2, updraft_type=utype)   #VERSION 2

    return Dictionary

In [ ]:
def SaveProfile(Dictionary,data_type):
    # print("Saving all profiles...")

    # Construct the output file path
    dir2 = dir + 'Project_Algorithms/Tracked_Profiles/job_out2/'
    output_file = dir2 + f"{data_type}_" + f"tracked_profiles_{res}_{t_res}_{Np_str}"
    output_file += f"_{job_id}.h5"

    # Write the entire dictionary to HDF5
    with h5py.File(output_file, "w") as h5f:
        for key, profile_data in Dictionary.items():
            h5f.create_dataset(key, data=profile_data)
            
    # print("Done saving.\n")

In [ ]:
##############################################
#RUNNING

In [ ]:
[start_slurm_job,end_slurm_job]=StartSlurmJobArray(num_jobs=num_jobs,num_slurm_jobs=num_slurm_jobs,ISRUN=True) #if ISRUN is False, then will not run using slurm_job_array

print(f"Running on Slurm_Jobs for Slurm_Job_Ids: {(start_slurm_job,end_slurm_job)}")

job_id_list=np.arange(start_slurm_job,end_slurm_job)
for job_id in job_id_list:
    if job_id % 1 == 0: print(f'current job_id = {job_id}\n')
    [start_job,end_job,index_adjust] = StartJobArray(job_id,num_jobs)
    print(f"Running Job_Array ID = {job_id}: {(start_job,end_job)}")

    #SLICING DATA
    parcel=parcel1.isel(xh=slice(start_job,end_job))
    apply_job_array_filter(start_job, end_job)

    #GETTING DATA AND PUTTING IN A DICTIONARY
    [Z,Y,X,parcel_z] = GetSpatialData(start_job,end_job) 

    #CALCULATING AND SAVING
    Dictionary=RunCalculation(Z,Y,X) #VERSION1: EACH VARIABLE SEPERATELY
    SaveProfile(Dictionary,data_type)

    #check_memory(globals())
    del Dictionary

In [ ]:
#########################################
#RECOMBINE SEPERATE JOB_ARRAYS AFTER
recombine=False #KEEP FALSE WHEN JOBARRAY IS RUNNING
# recombine=True

In [ ]:
def GetInputFile(data_type,job_id,dir2):
    #CALLING IN DATA
    input_file=dir2 + f"{data_type}_" + f"tracked_profiles_{res}_{t_res}_{Np_str}_{job_id}.h5"
    return input_file
def GetOutputFile(data_type,type1,type2,dir3):
    if type2==None:
        output_file=dir3 + f"{data_type}_" + f"{type1}_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
    else:
        output_file=dir3 + f"{data_type}_" + f"{type1}_{type2}_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
    return output_file
def Recombine(type1,type2,num_jobs):
    global variables
    print(f'recombining {type1}_{type2}')
    if type2==None:
        type2s = [f"{type1}"]
    else:
        type2s = [f"{type1}",f"{type2}"]
    types = ["ALL", "SHALLOW", "DEEP"]

    dir2=dir+'Project_Algorithms/Tracked_Profiles/job_out2/'
    dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
    
    #MAKING OUTPUT FILE PATH
    output_file=GetOutputFile(data_type,type1,type2,dir3)

    job_id=1; input_file=GetInputFile(data_type,job_id,dir2)
    with h5py.File(input_file, 'r') as f:
        all_keys = list(f.keys())
        vars_list = [k for k in all_keys if any(t2 in k for t2 in type2s)]

    
    #MAKING PROFILES DICTIONARY
    zhs = data1['zh'].values
    profiles = {}  # Store profiles for all variables
    for var in vars_list:
        profiles[var] = np.zeros((len(zhs), 3))  # column 1: var, column 2: counter, column 3: list of zhs
        profiles[var][:, 2] = zhs 

    for job_id in np.arange(1,num_jobs+1):
        if np.mod(job_id,20)==0: print(f"job_id = {job_id}")
    
        #CALLING IN DATA
        input_file=GetInputFile(data_type,job_id,dir2)
    
        #COMPILING PROFILES
        with h5py.File(input_file, 'r') as f:
            for var in vars_list:
                profiles[var][:,0:1+1]+=f[f'{var}'][:,0:1+1]
    
    #SAVING INTO FINAL FORM
    with h5py.File(output_file, 'w') as f:
        for var in profiles:
            profile_var = profiles[var]
            f.create_dataset(f'{var}', data=profile_var, compression="gzip")

In [ ]:
if recombine==True:
    Recombine(type1='CL',type2='nonCL',num_jobs=num_jobs)
    Recombine(type1='SBZ',type2='nonSBZ',num_jobs=num_jobs)
    Recombine(type1='ColdPool',type2=None,num_jobs=num_jobs)

In [ ]:
###########################
# #TESTING

In [ ]:
# job_id=50
# type1='CL';type2='nonCL'
# dir2 = dir + 'Project_Algorithms/Tracked_Profiles/job_out2/'
# output_file = dir2 + f"{data_type}_" + f"tracked_profiles_{res}_{t_res}_{Np_str}"
# output_file += f"_{job_id}.h5"
# key_list=[]
# with h5py.File(output_file, 'r') as h5f:
#     for key in h5f.keys():
#         # one=h5f["CL_ALL_profile_array_cloudy"][:]
#         two=h5f["CL_ALL_profile_array_cloudy"][:]

In [ ]:
# print(one==two)
# print(one[:,0]-two[:,0])
# plt.plot(two[:,0],two[:,2])